In [ ]:
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "발급받은 API 키"

In [ ]:
from json.decoder import JSONDecodeError
from openai import OpenAI
import os
import re
import json
import time

client = OpenAI()
DEBUG = False

# 기정의된 시스템 메시지
SYSTEM_MESSAGE = """

다음의 전제 지식과 지원 기록을 바탕으로 지원을 해주시기 바랍니다.
모든 대화에 대한 답변과 그 요약을 다음과 같은 JSON 형식으로 출력해주세요.

## 출력 (JSON 형식)
{{"content" : "(답변 내용)", "summary" : "(답변 요약 내용)" }}

## 대화의 예
USER->기계의 상태가 나쁩니다
AI->{{"content" : "어떤 기계에서 어떤 문제가 발생하고 있습니까?", "summary" : "기계의 종류와 문제 현상을 사용자에게 확인."}}

USER->감사합니다. (종료의 암시, 인사, 감사의 말 등도 포함)
AI->{{"content" : "천만에요. 언제든지 상담을 요청해 주세요.","summary" : "지원 종료."}}

## 전제 지식
  - 당신은 농기구 고장 진단의 전문가입니다.

## 지원 기록
  {support_context}

"""

def create_chat_response(user_text, support_context):
    for i in range(3):
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": SYSTEM_MESSAGE.format(support_context=support_context)
                },
                {
                    "role": "user",
                    "content": f"JSON 형식으로 출력해주세요: [{user_text}]+"
                }
            ],
        )

        try:
            # 응답으로부터 JSON 형식의 데이터만 추출.
            json_str = re.search(r'\{.*\}', response.choices[0].message.content).group(0)
            return json.loads(json_str)
        except JSONDecodeError:
            # 잘못된 JSON 데이터가 반환된 경우 재시도하고, 이 때 AI에게 주의를 던짐.
            if DEBUG:
                print("JSON Parse Error: ", response.choices[0].message.content)
            print("...AI의 출력에 오류가 발생했습니다. 재시도합니다......")
            user_text += '...응답 시 JSON 형식을 지켜 주세요.'
            time.sleep(1)
            continue
        except AttributeError:
            # JSON으로 변환할 수 없는 것들. (목록 등) 이 경우 값을 그대로 설정.
            if DEBUG:
                print("Attribute Error:", response.choices[0].message.content)
            return {
                "content": response.choices[0].message.content,
                "summary": response.choices[0].message.content,
                "error": "attribute error"
            }
    else:
        return []

support_context = ""
print("농기구에 대해 어려움이 있다면 답변해드립니다.\n무엇이든 물어보세요.\n")

while True:
    user_text = input("\n>")

    if user_text == "quit":
        break

    response = create_chat_response(user_text, support_context)

    if response:
        answer  = response['content']
        summary = response['summary']
    else:
        print("죄송합니다. 질문 방법을 바꿔주세요.")
        continue

    support_context += f"- user: {user_text}\n"
    support_context += f"- AI: {summary}\n\n"
    print("\n답변: ", answer)

    if DEBUG:
        print("요약: ", summary)
        print("response: ", response)